# Análisis para tienda online
## Objetivo:
Análisis de hipótesis para el aumento de ingresos
### Objetivos específicos:
* Priorizar las hipótesis
* Analizar el test A/B
    * Ingreso acumudalo por grupo
    * Número de pedidos por grupo
    * Ticket promedio por grupo y su diferencia relativa
    * Tasa de conversión para cada grupo
    * Identificar anomalías
    * Significancia estadística de la diferencia en la tasa de conversión y el ticket promedio para cada grupo


## Pasos a seguir
1. Priorizar las hipótesis
    * Aplicar framework ICE
    * Aplicar framework RICE

2. Análisis de test A/B
    * Limpieza de datos
    * Ingreso acumudalo por grupo
    * Número de pedidos por grupo
    * Ticket promedio por grupo y su diferencia relativa
    * Tasa de conversión para cada grupo
    * Identificar anomalías
    * Significancia estadística de la diferencia en la tasa de conversión y el ticket promedio para cada grupo

3. Toma de decisión sobre la prueba

## Diccionario de datos

**Hypotheses** (hipótesis)
* ``Hypotheses``: breves descripciones de las hipótesis
* ``Reach``: alcance del usuario, en una escala del uno a diez
* ``Impact``: impacto en los usuarios, en una escala del uno al diez
* ``Confidence``: confianza en la hipótesis, en una escala del uno al diez
* ``Effort``: los recursos necesarios para probar una hipótesis, en una escala del uno al diez. Cuanto mayor sea el valor Effort, más recursos requiere la prueba

**Orders** (pedidos realizados)
* ``transactionId``: identificador de pedido
* ``visitorId``: identificador del usuario que realizó el pedido
* ``date``: fecha del pedido
* ``revenue``: ingresos del pedido
* ``group``: el grupo del test A/B al que pertenece el usuario

**Visits** (registro de visitas)
* ``date``: la fecha
* ``group``: grupo de la prueba A/B
* ``visits``: el número de visitas en la fecha especificada en el grupo de pruebas A/B especificado